In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Keras
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

%matplotlib inline

In [2]:
# returns train, inference_encoder and inference_decoder models
def models(n_input, n_output, n_units):
    """
    Gets and prints the spreadsheet's header columns

    Parameters
    ----------
    n_input : str
        Length of input sequence, e.g number of integers
    n_output : bool, optional
        Length of output sequence, e.g number of integers
    n_units : bool, optional
        N umber of cells to create in the encoder and decoder models

    Returns
    -------
    model, encoder_model, decoder_model
        3 models, 
    """
    
    # training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    
    # training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    # inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model

In [4]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    """
    Gets and prints the spreadsheet's header columns

    Parameters
    ----------
    infenc: Encoder model used when making a prediction for a new source sequence
    infdec: Decoder model use when making a prediction for a new source sequence
    source:Encoded source sequence
    n_steps: Number of time steps in the target sequence
    cardinality: The cardinality of the output sequence, e.g. the number of features, words, or characters for each time step

    Returns
    -------
    array(output)
        target sequence
    """
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat

    return array(output)

In [39]:
from random import randint
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

In [42]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return np.random.randint(n_unique, size=length)

In [43]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [48]:
# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define target sequence
        target = sorted(source)
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)

    return array(X1), array(X2), array(y)

In [49]:
# configure problem
n_features = 50 
n_steps_in = 6
n_steps_out = 3
# generate a single source and target sequence
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

(1, 6, 50) (1, 6, 50) (1, 6, 50)
X1=[28, 27, 2, 21, 40, 26], X2=[0, 2, 21, 26, 27, 28], y=[2, 21, 26, 27, 28, 40]


In [ ]:
# Apply encoder-decoder LSTM model